In [2]:
import ifcopenshell
import csv

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")

# Name of the target storey
storey_name = "Plan 10"

# Find the target storey
target_storey = None
for storey in ifc_file.by_type("IfcBuildingStorey"):
    if storey.Name == storey_name:
        target_storey = storey
        break

if not target_storey:
    print(f"Storey with name '{storey_name}' was not found.")
else:
    print(f"Storey '{storey_name}' found. OID: {target_storey.id()}")

    # Initialize a list to store the results
    elements_to_walls = []

    # Function to find the hosting wall for an opening element
    def find_hosting_wall(opening_element):
        for rel_voids in ifc_file.by_type("IfcRelVoidsElement"):
            if rel_voids.RelatedOpeningElement == opening_element:
                return rel_voids.RelatingBuildingElement
        return None

    # Find all IfcDoors and IfcWindows in the target storey
    for rel_contained in target_storey.ContainsElements:
        for element in rel_contained.RelatedElements:
            if element.is_a("IfcDoor") or element.is_a("IfcWindow"):
                element_type = element.is_a()
                element_guid = element.GlobalId

                # Find the IfcRelFillsElement relationships
                target_opening = None
                for rel_fills in ifc_file.by_type("IfcRelFillsElement"):
                    if rel_fills.RelatedBuildingElement == element:
                        target_opening = rel_fills.RelatingOpeningElement
                        break

                if target_opening:
                    # Find the hosting wall
                    hosting_wall = find_hosting_wall(target_opening)
                    if hosting_wall:
                        wall_oid = hosting_wall.id()
                        print(f"{element_type} with GUID '{element_guid}' is hosted by wall OID: {wall_oid}")
                        elements_to_walls.append({
                            "element_type": element_type,
                            "element_guid": element_guid,
                            "wall_oid": wall_oid
                        })

    # Write the results to a CSV file
    with open('Output05_HostWindowsAndDoors.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=';')
        csvwriter.writerow(["Element Type", "Element GUID", "Wall OID"])
        for entry in elements_to_walls:
            csvwriter.writerow([entry["element_type"], entry["element_guid"], entry["wall_oid"]])

    print("Results have been saved to Output_Elements_HostingWalls.csv")

Storey 'Plan 10' found. OID: 118
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSdd' is hosted by wall OID: 49410
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSdc' is hosted by wall OID: 49972
IfcDoor with GUID '0aNC3YsFzEZf8uB9g1nSdw' is hosted by wall OID: 35804
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSdz' is hosted by wall OID: 49639
IfcDoor with GUID '0aNC3YsFzEZf8uB9g1nSdy' is hosted by wall OID: 35742
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSdn' is hosted by wall OID: 49751
IfcDoor with GUID '0aNC3YsFzEZf8uB9g1nSds' is hosted by wall OID: 37423
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nScC' is hosted by wall OID: 37978
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSc3' is hosted by wall OID: 37978
IfcDoor with GUID '0aNC3YsFzEZf8uB9g1nSce' is hosted by wall OID: 39201
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nScl' is hosted by wall OID: 50492
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nSck' is hosted by wall OID: 50492
IfcWindow with GUID '0aNC3YsFzEZf8uB9g1nScW' is hosted by wall OID: 37978
IfcWindow wit